In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [36]:
url = "http://www.espn.com/mens-college-basketball/teams"

In [37]:
response = requests.get(url)

In [38]:
espn = BeautifulSoup(response.text, 'lxml')

In [48]:
all_tables = wiki.find_all("ul")

In [49]:
print(all_tables[0].prettify)

<bound method Tag.prettify of <ul class="medium-logos"><li class="first"><span>
<a href="/ncb/teams/stats?teamId=399">Stats</a> | <a href="/ncb/teams/schedule?teamId=399">Schedule</a> | <a href="/ncb/teams/roster?teamId=399">Roster</a> </span>
</li>
<li class="alt"><h5><a class="bi" href="http://www.espn.com/mens-college-basketball/team/_/id/2066/binghamton-bearcats">Binghamton</a></h5><span>
<a href="/ncb/teams/stats?teamId=2066">Stats</a> | <a href="/ncb/teams/schedule?teamId=2066">Schedule</a> | <a href="/ncb/teams/roster?teamId=2066">Roster</a> </span>
</li>
<li class=""><h5><a class="bi" href="http://www.espn.com/mens-college-basketball/team/_/id/42/hartford-hawks">Hartford</a></h5><span>
<a href="/ncb/teams/stats?teamId=42">Stats</a> | <a href="/ncb/teams/schedule?teamId=42">Schedule</a> | <a href="/ncb/teams/roster?teamId=42">Roster</a> </span>
</li>
<li class="alt"><h5><a class="bi" href="http://www.espn.com/mens-college-basketball/team/_/id/311/maine-black-bears">Maine</a></h5

In [113]:
all_links = all_tables[1].findAll("a", href = True)
team_links = all_links[3]["href"]
team_links

'http://www.espn.com/mens-college-basketball/team/_/id/151/east-carolina-pirates'

## Time to scrape the urls

In [116]:
rosterUrls = []

for conference in all_tables:
            
    rows = conference.findAll("li")    
    for links in rows:
        roster_link = links.findAll("a", href = True, text = "Roster")
        if (len(roster_link) > 0):
            roster_link_clean = "http://www.espn.com" + roster_link[0]["href"]
            rosterUrls.append(roster_link_clean)
            
print(len(rosterUrls))

338


In [118]:
rosterUrls[0:10]

['http://www.espn.com/ncb/teams/roster?teamId=399',
 'http://www.espn.com/ncb/teams/roster?teamId=2066',
 'http://www.espn.com/ncb/teams/roster?teamId=42',
 'http://www.espn.com/ncb/teams/roster?teamId=311',
 'http://www.espn.com/ncb/teams/roster?teamId=160',
 'http://www.espn.com/ncb/teams/roster?teamId=2619',
 'http://www.espn.com/ncb/teams/roster?teamId=2349',
 'http://www.espn.com/ncb/teams/roster?teamId=2378',
 'http://www.espn.com/ncb/teams/roster?teamId=261',
 'http://www.espn.com/ncb/teams/roster?teamId=2132']

### Now I have all the team roster URLs. I can go into all the teams and collect every player's information. 

In [182]:
teams = []
players = []
positions = []
heights = []
weights = []
classes = []

for roster in rosterUrls:
    team = requests.get(roster)
    team_text = BeautifulSoup(team.text, 'lxml')
    team_name = team_text.findAll(True, {"class": ["stathead"]})[0].text
    player_rows = team_text.findAll(True, {"class": ["oddrow", "evenrow"]})
    for player in player_rows:        
        player_name = player.findAll("a", href = True)[0].text
        pos = player.findAll("td")[2].text
        height = player.findAll("td")[3].text.replace("-", ".")
        weight = player.findAll("td")[4].text
        class_ = player.findAll("td")[5].text
        
        teams.append(team_name)
        players.append(player_name)
        positions.append(pos)
        heights.append(height)
        weights.append(weight)
        classes.append(class_)    
        
print(len(teams))
print(len(players))
print(len(positions))
print(len(heights))
print(len(weights))
print(len(classes))

5255
5255
5255
5255
5255
5255


In [183]:
df = pd.DataFrame(teams, columns = ["TeamName"])
df["Player"] = players
df["Position"] = positions
df["Height"] = heights
df["Weights"] = weights
df["Class"] = classes

df.head(30)

,TeamName,Player,Position,Height,Weights,Class
0,Albany Great Danes,Xavier Cochran,G,6.6,195,SO
1,Albany Great Danes,Costa Anderson,G,6.2,195,SR
2,Albany Great Danes,Ahmad Clark,G,6.1,175,SO
3,Albany Great Danes,Brent Hank,F,6.10,265,FR
4,Albany Great Danes,Cameron Healy,G,6.3,190,FR
5,Albany Great Danes,Devonte Campbell,F,6.6,220,JR
6,Albany Great Danes,David Nichols,G,6.0,190,JR
7,Albany Great Danes,Adam Lulka,F,6.8,235,FR
8,Albany Great Danes,Nick Fruscio,G,5.10,155,SO
9,Albany Great Danes,Joe Cremo,G,6.4,185,JR


In [184]:
df.to_csv("datasets/ncaa_2018_players.csv", encoding='utf-8')

In [187]:
testing = pd.read_csv("datasets/ncaa_2018_players.csv").reset_index(drop = True)
testing.head()

,Unnamed: 0,TeamName,Player,Position,Height,Weights,Class
0,0,Albany Great Danes,Xavier Cochran,G,6.6,195,SO
1,1,Albany Great Danes,Costa Anderson,G,6.2,195,SR
2,2,Albany Great Danes,Ahmad Clark,G,6.1,175,SO
3,3,Albany Great Danes,Brent Hank,F,6.10,265,FR
4,4,Albany Great Danes,Cameron Healy,G,6.3,190,FR


## Got it!